In [3]:
import pandas as pd
import numpy as np
import sqlite3
import os
pd.set_option("display.max_columns",75)


In [4]:
conn = sqlite3.Connection("../artifacts/data/db.sqlite3")

dat = pd.read_sql('select * from RAW',
con=conn)

In [5]:
## FEATURE FUNCS

def strip_and_make_0_float(x: str) -> float:
    """
    Special purpose function for saving only a float value found at front of a string in stats data.

    Args:
        x (str): string that contained a float followed by multiple pieces of metadata space-separated

    Returns:
        float: value pulled from string
    """
    return x.str.split(expand=True)[0].astype(float)

def featurize(table: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a table of stats data and returns additional feature columns for ML processes.

    Args:
        table (pd.DataFrame): Table of Raws

    Returns:
        pd.DataFrame: DF with features
    """
    table[['w','l']]=table['rec'].str.split(expand=True)[0].str.split("–",expand=True).astype(int)
    table['win_perc'] = table['w']/(table['w']+table['l'])

    table['tourney'] = np.where(
        table['team'].str.contains('CHAMPS'),'CHAMPS',
        np.where(
        table['team'].str.contains('Finals'),'Finals',
        np.where(
        table['team'].str.contains('Final Four'),'Final Four',
        np.where(
        table['team'].str.contains('Elite Eight'),'Elite Eight',
        np.where(
        table['team'].str.contains('Sweet Sixteen'),'Sweet Sixteen',
        np.where(
        table['team'].str.contains('R32'),'R32',
        np.where(
        table['team'].str.contains('R64'),'R64',
        np.where(
        table['team'].str.contains('R68'),'R68',
        'no tourney'
        )
        )
        )
        )
        )
        )
        )
    )

    table['team']=table['team'].str.split(",",expand=True)[0]
    table['team'] = table['team'].str.replace(" seed","").apply(
        lambda x: ''.join([i for i in x if (i.isalpha())|(i==" ")])
    ).str.strip()

    tourney_values = {
        'no tourney':128,
        'R68':68,
        'R64':64,
        'R32':32,
        'Sweet Sixteen':16,
        'Elite Eight':8,
        'Final Four':4,
        'Finals':2,
        'CHAMPS':1,
    }

    table['OUTCOME'] = table['tourney'].map(tourney_values)

    table['adjoe'] = strip_and_make_0_float(table['adjoe'])
    table['adjde'] = strip_and_make_0_float(table['adjde'])
    table['barthag'] = strip_and_make_0_float(table['barthag'])
    table['efg_pct'] = strip_and_make_0_float(table['efg_pct'])
    table['efgd_pct'] = strip_and_make_0_float(table['efgd_pct'])
    table['tor'] = strip_and_make_0_float(table['tor'])
    table['tord'] = strip_and_make_0_float(table['tord'])
    table['orb'] = strip_and_make_0_float(table['orb'])
    table['drb'] = strip_and_make_0_float(table['drb'])
    table['ftr'] = strip_and_make_0_float(table['ftr'])
    table['ftrd'] = strip_and_make_0_float(table['ftrd'])
    table['2p_pct'] = strip_and_make_0_float(table['2p_pct'])
    table['2pd_pct'] = strip_and_make_0_float(table['2pd_pct'])
    table['3p_pct'] = strip_and_make_0_float(table['3p_pct'])
    table['3pd_pct'] = strip_and_make_0_float(table['3pd_pct'])
    table['3pr'] = strip_and_make_0_float(table['3pr'])
    table['3prd'] = strip_and_make_0_float(table['3prd'])
    table['adj_t'] = strip_and_make_0_float(table['adj_t'])
    table['wab'] = strip_and_make_0_float(table['wab'])

    conferences = ['WCC', 'Amer', 'B12', 'ACC', 'SEC', 'BE', 'P12', 'B10', 'MWC',
        'MVC', 'A10', 'OVC', 'CUSA', 'AE', 'SC', 'WAC', 'Sum', 'CAA',
        'MAAC', 'MAC', 'Ivy', 'ASun', 'Pat', 'SB', 'BW', 'BSth', 'BSky',
        'NEC', 'Horz', 'SWAC', 'MEAC', 'Slnd']

    for c in conferences:
        table[c] = np.where(table['conf']==c,1,0)

    return table

In [6]:
features = featurize(dat)


In [7]:
features

,rk,team,conf,g,rec,adjoe,adjde,barthag,efg_pct,efgd_pct,tor,tord,orb,drb,ftr,ftrd,2p_pct,2pd_pct,3p_pct,3pd_pct,3pr,3prd,adj_t,wab,year,load_date,w,l,win_perc,tourney,OUTCOME,WCC,Amer,B12,ACC,SEC,BE,P12,B10,MWC,MVC,A10,OVC,CUSA,AE,SC,WAC,Sum,CAA,MAAC,MAC,Ivy,ASun,Pat,SB,BW,BSth,BSky,NEC,Horz,SWAC,MEAC,Slnd
0,1,Kansas,B12,33,30–3 13–3,121.0,85.6,0.9816,56.3,44.8,18.7,22.9,38.0,29.0,37.5,30.8,54.8,40.9,39.9,34.0,29.2,38.1,69.5,9.9,2008,2024-03-18,30,3,0.909091,CHAMPS,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Memphis,CUSA,34,33–1 16–0,113.3,83.8,0.9697,53.0,42.5,17.2,23.3,37.8,29.3,38.6,32.7,53.3,41.6,35.0,29.8,36.7,28.7,70.7,8.9,2008,2024-03-18,33,1,0.970588,Finals,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,UCLA,P10,33,30–3 16–2,116.4,86.9,0.9664,52.3,48.0,18.8,22.7,39.2,25.9,38.0,25.7,52.9,46.8,33.8,34.2,28.1,27.9,66.2,10.8,2008,2024-03-18,30,3,0.909091,Final Four,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Wisconsin,B10,33,29–4 16–2,112.2,84.9,0.9609,50.6,43.1,19.4,22.2,36.2,28.5,39.3,25.4,49.2,41.3,35.9,31.1,31.1,32.8,63.5,8.3,2008,2024-03-18,29,4,0.878788,Sweet Sixteen,16,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,Duke,ACC,32,27–5 13–3,117.2,88.8,0.9607,54.1,47.6,18.2,24.9,34.3,33.5,40.4,32.0,51.8,47.1,38.4,32.8,39.1,24.8,73.7,8.9,2008,2024-03-18,27,5,0.843750,R32,32,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5602,358,Stonehill,NEC,30,3–27 2–14,90.4,113.9,0.0658,46.7,52.7,19.5,16.6,22.5,31.0,22.6,29.4,47.9,51.7,30.1,35.9,47.0,46.2,69.1,-22.0,2024,2024-03-18,3,27,0.100000,no tourney,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5603,359,St Francis PA,NEC,28,6–22 3–13,93.2,117.5,0.0649,47.2,53.0,21.2,17.1,32.9,31.3,32.6,35.4,45.7,52.9,33.2,35.4,35.1,37.1,66.4,-18.6,2024,2024-03-18,6,22,0.214286,no tourney,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5604,360,IUPUI,Horz,29,3–26 2–18,92.5,116.9,0.0630,46.5,58.2,21.3,18.5,30.0,35.5,33.2,33.4,49.5,59.0,24.7,38.0,24.3,37.5,68.3,-21.6,2024,2024-03-18,3,26,0.103448,no tourney,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5605,361,Coppin St,MEAC,29,2–27 1–13,84.7,110.1,0.0468,42.1,51.3,22.9,21.8,27.0,38.6,31.1,38.3,41.9,51.0,28.3,34.5,34.4,37.6,67.3,-22.9,2024,2024-03-18,2,27,0.068966,no tourney,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
